In [107]:
import os
os.chdir("/Users/n0362948/desktop/Bread")

In [108]:
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
import scipy.stats as stats

intellicron_prequals=pd.read_csv('/Users/n0362948/desktop/Bread/intellicron_prequals.csv')
prequals=pd.read_csv('/Users/n0362948/desktop/Bread/prequals.csv')

In [109]:
intellicron_prequals['group']='test'

In [110]:
date_min=intellicron_prequals['assignment_date'].min()
date_max=intellicron_prequals['assignment_date'].max()

In [111]:
temp=pd.merge(prequals,intellicron_prequals,how='left', on='prequal_id')

In [112]:
temp['Total']=1

In [113]:
temp['checkout']=temp['checkout_id'].apply(lambda x: 0 if x is np.nan else 1 )

In [114]:
def group(row):
    if row['group']=='test': 
        return 'test'
    else:
        return 'control'
    
temp['group']=temp.apply(group, axis=1)

In [115]:
split=temp.loc[(temp['prequal_date']<=date_max) & (temp['prequal_date']>=date_min)]

In [116]:
clean_split=split[['prequal_id','checkout_id','prequal_date','completed_prequal','approved','checkout','assignment_date','group','Total']]

In [117]:
clean_split.shape, clean_split.columns

((18185, 9),
 Index(['prequal_id', 'checkout_id', 'prequal_date', 'completed_prequal',
        'approved', 'checkout', 'assignment_date', 'group', 'Total'],
       dtype='object'))

In [118]:
clean_pivot=clean_split[['completed_prequal', 'approved',  'group','checkout', 'Total']].groupby('group').agg('sum')

In [119]:
completed_prequal=clean_split.pivot_table(values='completed_prequal', index='group', aggfunc=np.sum)
completed_prequal['total']=clean_split.pivot_table(values='completed_prequal', index='group', aggfunc=lambda x:len(x))
completed_prequal['completion_prequal_rate']=clean_split.pivot_table(values='completed_prequal', index='group')


In [120]:
completed_prequal.reset_index()

,group,completed_prequal,total,completion_prequal_rate
0,control,5026,9690,0.518679
1,test,4658,8495,0.548323


In [121]:
completed_checkout=clean_split.pivot_table(values='checkout', index='group', aggfunc=np.sum)
completed_checkout['total']=clean_split.pivot_table(values='checkout', index='group', aggfunc=lambda x:len(x))
completed_checkout['completion_prequal_rate']=clean_split.pivot_table(values='checkout', index='group')

In [122]:
completed_checkout.reset_index()

,group,checkout,total,completion_prequal_rate
0,control,2543,9690,0.262436
1,test,2307,8495,0.271572


In [123]:
### just to test if it works
clean_pivot.loc['control']['Total']


9690

In [124]:
clean_pivot['prequal_rate']=clean_pivot['completed_prequal']/clean_pivot['Total']
clean_pivot['approved_rate']=clean_pivot['approved']/clean_pivot['Total']
clean_pivot['checkout_rate']=clean_pivot['checkout']/clean_pivot['Total']

In [125]:
clean_pivot

,completed_prequal,approved,checkout,Total,prequal_rate,approved_rate,checkout_rate
group,,,,,,,
control,5026,3371,2543,9690,0.518679,0.347884,0.262436
test,4658,3088,2307,8495,0.548323,0.363508,0.271572


In [126]:
def prop_test(convert_control, size_control, convert_test, size_test):
  
    p_a = convert_control / size_control
    p_b = convert_test / size_test
    p_pooled = (convert_control + convert_test) / (size_control + size_test)
    var = p_pooled * (1 - p_pooled) * (1 / size_control + 1 / size_test)
    zscore = np.abs(p_b - p_a) / np.sqrt(var)
    one_side = 1 - stats.norm(loc = 0, scale = 1).cdf(zscore)
    pvalue = one_side * 2
    return zscore, pvalue

In [141]:
### this is data for completed prequal out of all test data--> to calculate p value

convert_control = 5026
size_control = 9690
convert_test = 4658
size_test = 8495

zscore, pvalue = prop_test(convert_control, size_control, convert_test, size_test)
print('zscore = {:.3f}, pvalue = {:.6f}'.format(zscore, pvalue))

zscore = 3.997, pvalue = 0.000064


In [142]:
def confidence_interval(convert_control, size_control, convert_test, size_test, significance = 0.01):
    """
    A/B test for two proportions;
    given a success a trial size of group A and B compute
    its confidence interval;
    resulting confidence interval 
    
    Parameters
    ----------
    success_a, success_b : int
        Number of successes in each group
        
    size_a, size_b : int
        Size, or number of observations in each group
        
    significance : float, default 0.05
        Often denoted as alpha. Governs the chance of a false positive.
        A significance level of 0.05 means that there is a 5% chance of
        a false positive. In other words, our confidence level is
        1 - 0.05 = 0.95
        
    Returns
    -------
    prop_diff : float
        Difference between the two proportion
    
    confint : 1d ndarray
        Confidence interval of the two proportion test
    """
    p_a = convert_control / size_control
    p_b = convert_test / size_test
    var = p_a * (1 - p_a) / size_control + p_b * (1 - p_b) / size_test
    se = np.sqrt(var)
    
    # z critical value
    confidence = 1 - significance
    z = stats.norm(loc = 0, scale = 1).ppf(confidence + significance / 2)

    # standard formula for the confidence interval
    # point-estimtate +- z * standard-error
    prop_diff = p_b - p_a
    confint = prop_diff + np.array([-1, 1]) * z * se
    return prop_diff, confint

In [143]:
### this is data for completed prequal out of all test data--> to calculate confidence interval

prop_diff, confint = confidence_interval(convert_control, size_control, convert_test, size_test)
print('estimate difference:', prop_diff)
print('confidence interval:', confint)

estimate difference: 0.029643492104564628
confidence interval: [ 0.01055488  0.04873211]


In [137]:
### this is data for approved out of all test data--> to calculate p value and confidence interval 

convert_control2 = 3371
size_control2 = 9690
convert_test2 = 3088
size_test2 = 8495

zscore, pvalue = prop_test(convert_control2, size_control2, convert_test2, size_test2)
print('zscore = {:.3f}, pvalue = {:.6f}'.format(zscore, pvalue))

zscore = 2.196, pvalue = 0.028060


In [138]:
prop_diff, confint = confidence_interval(convert_control2, size_control2, convert_test2, size_test2)
print('estimate difference:', prop_diff)
print('confidence interval:', confint)

estimate difference: 0.015623528925835684
confidence interval: [ 0.00167499  0.02957207]


In [139]:
### this is to calculate the p value and conidence interval for checkout rate

convert_control3 = 2543
size_control3 = 9690
convert_test3 = 2307
size_test3 = 8495

zscore, pvalue = prop_test(convert_control3, size_control3, convert_test3, size_test3)
print('zscore = {:.3f}, pvalue = {:.6f}'.format(zscore, pvalue))

zscore = 1.390, pvalue = 0.164553


In [140]:
prop_diff, confint = confidence_interval(convert_control3, size_control3, convert_test3, size_test3)
print('estimate difference:', prop_diff)
print('confidence interval:', confint)

estimate difference: 0.009136012138506788
confidence interval: [-0.00375546  0.02202749]
